In [ ]:
from ipynb.fs.full.Plot_Noro import *

In [ ]:
import pandas as pd
from datetime import datetime as dt
import json, time, os

from bokeh.io import output_notebook, show, curdoc
from bokeh.document import Document

from bokeh.models import CheckboxButtonGroup, CheckboxGroup, MultiSelect, RadioButtonGroup,\
        Select, Slider, Panel, Tabs, TextInput, Paragraph, Div, Button, Dropdown,\
        DateRangeSlider, Column, Row

### Dashboard base class

In [ ]:
def get_slider_state(doc, item_name):
    slider = doc.get_model_by_name(item_name)
    start_date = slider.value[0]
    end_date = slider.value[1]
    if isinstance(start_date, int) or isinstance(start_date, float):
        start_date = dt.fromtimestamp(start_date/1000).strftime("%d-%m-%Y")
        end_date = dt.fromtimestamp(end_date/1000).strftime("%d-%m-%Y")
    else:
        start_date = pd.to_datetime(start_date).strftime("%d-%m-%Y")
        end_date = pd.to_datetime(end_date).strftime("%d-%m-%Y")
        
    return dict(start_date=start_date, end_date=end_date, type='slider', name=slider.name)

def get_checkbox_state(doc, item_name):
    checkbox = doc.get_model_by_name(item_name)
    values = [checkbox.labels[a] for a in checkbox.active]
    return dict(values=values, type='checkbox', name='checkbox')

def get_radiobuttongroup_state(doc, item_name):
    radiobuttongroup = doc.get_model_by_name(item_name)
    value = radiobuttongroup.labels[radiobuttongroup.active]
    return dict(values=value, type='radiobuttongroup', name='radiobuttongroup')

class Dash_base():
    """
    Base class for the dashboards
    """

    # Items to show
    _layout_items = {'controls':{}, 'plot':None, 'api_url':Div()}
    
    # dataframe
    df = pd.DataFrame()
    
    # categories
    dict_categories = {}    
    
    #own curdoc
    #doc = Document()
    doc = curdoc()
    
    # the url for referencing the custom set plot
    api_url_div = ""
    label = "label"
    endpoint_name = "endpoint"
    
    # Dashboard or API. The latter is without control items
    plot_mode = "dashboard"
    
    # import logging
# logging.basicConfig(filename='/tmp/app.log', filemode='w', format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.DEBUG)
# logger = logging.getLogger(__name__)
    
    @property
    def endpoint(self):
        api_url_prefix = os.getenv("NB_URL")
        api_host = os.getenv("NB_HOST")
        endpoint = os.path.join(api_host, api_url_prefix, "report")
        return endpoint
        
    def __init__(self, f_dataframe, f_categories=""):
        self.df = pd.read_feather(f_dataframe)   
        #logging.debug('Init class Dash')
        
        with open(f_categories, 'r') as f:
            self.dict_categories = json.loads(f.read())

        # del unknown
        self.df = self.df[self.df['ORF1_sub']!='Unknown']

        
    def on_click_plot(self, new):   
        self.set_api_url()  
    
    def set_controls(self):
        controls = {}
        self._layout_items['controls'] = controls
        
    def init_plot(self):
        item_to_be_treplaced = Div(text="Placeholder for the plot")
        self._layout_items['plot'] = item_to_be_treplaced

    def create_api_url(self):
        self._layout_items['api_url'] =  Div(text="<h3>Placeholder</h3>", name='endpoint_%s'%self.label)
    
    def set_api_url(self):
        controls = self._layout_items['controls']
        
        fields = []
        for item_name in controls.keys():
            control = controls[item_name]
            if isinstance(control, DateRangeSlider):
                values =  get_slider_state(self.doc, item_name)
                fields.append("{name}={start_date},{end_date}".format(**values))
            if isinstance(control, CheckboxGroup):
                result =  get_checkbox_state(self.doc, item_name)
                newfield = "{0}={1}".format(item_name, ",".join(result['values']) if result['values'] else 'all')
                fields.append(newfield)
            if isinstance(control, RadioButtonGroup):
                result =  get_radiobuttongroup_state(self.doc, item_name)
                newfield = "{0}={1}".format(item_name, result['values'])
                fields.append(newfield)
        
        api_url = os.path.join(self.endpoint, "%s?"%self.endpoint_name+"&".join(fields))
        prev_div = self.doc.get_model_by_name('endpoint_%s'%self.label)
        self.api_url_div = Div(text="<h3>%s</h3>"%api_url, name='endpoint_%s'%self.label)
        prev_div.text = "<h3>%s</h3>"%api_url
        return self.api_url_div
        
        
    def set_layout(self):
        self.doc.add_root(Column())
        layout = []
        if 'controls' in self._layout_items.keys():
            layout.extend([self._layout_items['controls'][kc] for kc in self._layout_items['controls'].keys()])
            if self.plot_mode=='api':
                for kc in self._layout_items['controls'].keys():
                    self._layout_items['controls'][kc].visible=False 
        layout.append(self._layout_items['plot'])
        if 'api_url' in self._layout_items.keys():
            layout.append(self._layout_items['api_url'])
            if self.plot_mode=='api':
                self._layout_items['api_url'].visible=False
        self.doc.roots[-1].children.extend(layout)
        self.set_api_url()


    def get_state_from_url(self, field_dict={}, test_url=""):
        if test_url:
            fields = url.split(self.endpoint)[1]
            if fields[0]=="/":
                fields = fields[1:]
            if fields[0]=="?":
                fields = fields[1:]
            field_dict = {f.split("=")[0]:[f.split("=")[1]] for f in fields.split("&")}
        
        controls = self._layout_items['controls']
        for item_name in field_dict.keys():
            control = controls[item_name]
            if isinstance(control, DateRangeSlider):
                control_item = self.doc.get_model_by_name(item_name)
                new_values = field_dict[item_name][0].split(",")
                new_start_date = pd.to_datetime(new_values[0], dayfirst=True)
                new_end_date = pd.to_datetime(new_values[1], dayfirst=True)
                self.set_control_state(control_item, start_date=new_start_date, end_date=new_end_date)

            if isinstance(control, CheckboxGroup):
                control_item = self.doc.get_model_by_name(item_name)
                new_active_labels = field_dict[item_name][0].split(",")
                self.set_control_state(control_item, labels=new_active_labels)
                
            if isinstance(control, RadioButtonGroup):
                control_item = self.doc.get_model_by_name(item_name)
                new_active_value = field_dict[item_name][0]
                self.set_control_state(control_item, value=new_active_value)    
                
    def set_control_state(self, item, **kwargs):
        if isinstance(item, DateRangeSlider):
            item.value = (kwargs['start_date'], kwargs['end_date'])
        if isinstance(item, CheckboxGroup):
            if kwargs['labels'] != ['all']:
                for l in kwargs['labels']:
                    item.active.append(item.labels.index(l))
        if isinstance(item, RadioButtonGroup):
            #item.active = int(kwargs['value'])
            item.active = item.labels.index(kwargs['value'])
        
    def show(self):
         lay=self.doc.roots[0]
#         self.doc.remove_root(self.doc.roots[0])
         return show(lay)        
      

In [ ]:
class Dash(Dash_base):
    """
    An intermediate class with shortcut functions and callbacks
    """
    def set_controls(self):
        # 
        controls = {}
    
        # checkbox for countries
        c_choices = list(self.df.Country.unique())
        c_choices.sort()
        checkbox_group = CheckboxGroup(labels=c_choices, active=[], inline=True, name='country_checkbox%s'%self.label)
        controls['country_checkbox%s'%self.label] = checkbox_group

        # checkbox for genotype
        g_choices = list(self.df['Genus-Genogroup'].unique())
        g_choices.sort()
        radio_group = RadioButtonGroup(labels=g_choices, active=0, name='genotype_choice%s'%self.label)
        controls['genotype_choice%s'%self.label] = radio_group
        
        # time slider
        dates = self.dict_categories['dates']
        min_date = "01-01-1990" #self.df['Sample Date'].min()
        max_date = self.df['Sample Date'].max()
        date_range_slider = DateRangeSlider(title="Date Range: ", start=min_date, end=max_date, value=(min_date, max_date), step=1, name='date_slider%s'%self.label)
        controls['date_slider%s'%self.label] = date_range_slider
        
        # button to replot
        b_choose = Button(label="Plot", button_type="success", name="button%s"%self.label)
        b_choose.on_click(self.on_click_plot)
        controls['submit_%s'%self.label] = b_choose
    
        #
        self._layout_items['controls'] = controls 
        
    def on_click_plot(self, new):   
        
        # Get the state of control elements
        BC = self.plot_type(self.df)
        
        # Date Slider
        newtime = get_slider_state(self.doc, 'date_slider%s'%self.label)
        BC.date_filter = [newtime['start_date'], newtime['end_date']]
        #logging.debug('start: %s, end: %s'%(start, end))

        # Country checkbox
        BC.selected_countries = get_checkbox_state(self.doc, 'country_checkbox%s'%self.label)['values']    
        
        # Genotype checkbox
#         genotype = 'Caliciviridae Norovirus GI'
        ## CAVEAT only one genotype at once
        BC.selected_genotype = get_radiobuttongroup_state(self.doc, 'genotype_choice%s'%self.label)['values']
        
        # ORF type radio button
        BC.selected_orf_type = 'ORF1_sub'
        
        # ORF subtype checkbox
        orf_subtypes = []

        BC.filter_df()
        pie_plot = BC.create_plot()
        
        e = self.doc.get_model_by_name('plot_div_%s'%self.label)
        e.children=[pie_plot]
        self.set_api_url()
        
    def do_all(self):
        #self.label = np.random.randint(10000)
        self.set_controls()
        self.init_plot()
        self.create_api_url()
        self.set_layout()
        self.set_api_url()

In [ ]:
class Dash_pie(Dash):
    """
    Class for pie charts
    
    :param plot_types: country_pie, outbreak_setting_pie_chart
    """
    
    plot_type = country_pie
    plot_mode = 'dashboard'
    selected_genotype = 'Caliciviridae Norovirus GI'
  
    def init_plot(self):
        BC = self.plot_type(self.df)
        BC.filter_df()
        pie_plot = BC.create_plot()
        pie_plot.tags = ['plott']
        pie_plot.name='plot_div_%s'%self.label
        #item_to_be_treplaced = Row(pie_plot, tags=['plott'], name='plot_div_%s'%self.label)
        item_to_be_treplaced = pie_plot
        self._layout_items['plot'] = item_to_be_treplaced
        
    
        

In [ ]:
class Dash_bar(Dash):
    """
    Class for bar charts
    
    :param plot_types: orf_subtype_bar_chart, factor_bar_chart
    """

    plot_type = orf_subtype_bar_chart
    plot_mode = 'dashboard'
    selected_genotype = 'Caliciviridae Norovirus GI'
       
    def init_plot(self):
        BC = self.plot_type(self.df)
        #BC.selected_genotype = 'Caliciviridae Norovirus GII'
        BC.filter_df()        
        pie_plot = BC.create_plot()
        pie_plot.tags = ['plott']
        pie_plot.name='plot_div_%s'%self.label
        #pie_plot = create_bar(df=tdf)
        #item_to_be_treplaced = Row(pie_plot, tags=['plott'], name='plot_div_%s'%self.label)
        item_to_be_treplaced = pie_plot
        self._layout_items['plot'] = item_to_be_treplaced
        

In [ ]:
# output_notebook()

# while len(curdoc().roots)>0:
#     curdoc().remove_root(curdoc().roots[0])

# D1 = Dash_pie('noronet_all_clean.fr', "categories_all.json")
# #D1.plot_type = country_pie
# #D1.plot_type = factor_bar_chart
# D1.plot_type = factor_bar_rev_chart
# D1.label = "pie" #np.random.randint(10000)

# D1.do_all()

# # D2.set_controls()
# # D2.init_plot()

# # D2.create_api_url()
# # D2.set_layout('dashboard')
# # #D.set_layout('api')
# # D2.set_api_url()
# #print(D1._layout_items)
# #url="https://kooplex-deploy.elte.hu/notebook/jegesm-retek-jegesm/report/country_pie?country_checkbox_cpie=all&genotype_choice_cpie=0&date_slider_cpie=01-01-1990,09-12-2019"
# # #D.get_state_from_url({'country_checkbox_pie': ['HUNGARY,GERMANY'], 'date_slider_pie': ['02-01-2018,02-12-2019']})
# D1.get_state_from_url({'country_checkbox_pie': ['all'], 'date_slider_pie': ['01-01-2015,31-12-2018'],'genotype_choice_pie': ['Caliciviridae Norovirus GI']})
# #D1.get_state_from_url({'country_checkbox_pie': ['all'], 'date_slider_pie': ['17-04-2018,02-12-2019'],'genotype_choice_pie': [0]})
# D1.on_click_plot('t')

# D1.show()

In [ ]:
D0 = Dash_bar('noronet_all_clean.fr', "categories_all.json")
D0.plot_type = country_pie
D0.endpoint_name = 'country_pie'
D0.label = 'cp'
D0.do_all()

D1 = Dash_pie('noronet_all_clean.fr', "categories_all.json")
D1.plot_type = outbreak_setting_pie_chart
D1.endpoint_name = 'outbreak_setting_pie_chart'
D1.label = 'op'
D1.do_all()

D2 = Dash_bar('noronet_all_clean.fr', "categories_all.json")
D2.plot_type = orf_subtype_bar_chart
D2.endpoint_name = 'orf_subtype_bar_chart'
D2.label = 'ob'
D2.do_all()

D3 = Dash_bar('noronet_all_clean.fr', "categories_all.json")
D3.plot_type = factor_bar_chart
D3.endpoint_name = 'factor_bar_chart'
D3.label = 'fb'
D3.do_all()

D32 = Dash_bar('noronet_all_clean.fr', "categories_all.json")
D32.plot_type = factor_bar_rev_chart
D32.endpoint_name= 'factor_bar_rev_chart'
D32.label = 'fbr'
D32.do_all()





# D1.label = np.random.randint(10000)
# D1.set_controls()
# D1.init_plot()

# D1.create_api_url()
# D1.set_layout('dashboard')
# D1.set_api_url()

In [ ]:
tab0 = Panel(child=D0.doc.roots[0],title="By Country Pie chart")
D0.doc.remove_root(D0.doc.roots[0])

tab1 = Panel(child=D1.doc.roots[0],title="Outbreak Settings Pie chart")
D1.doc.remove_root(D1.doc.roots[0])

tab2 = Panel(child=D2.doc.roots[0],title="By ORF type Bar chart")
D2.doc.remove_root(D2.doc.roots[0])

tab3 = Panel(child=D3.doc.roots[0],title="Factor Bar chart")
D3.doc.remove_root(D3.doc.roots[0])
tab32 = Panel(child=D32.doc.roots[0],title="Factor Bar Reverse chart")
D32.doc.remove_root(D32.doc.roots[0])

tabs = Tabs(tabs=[ tab0, tab1, tab2, tab3, tab32])
curdoc().add_root(tabs)

In [ ]:
#pie_plots = [country_pie, outbreak_setting_pie_chart]
#bar_plots = [orf_subtype_bar_chart, factor_bar_chart]


In [ ]:
show(tabs)

### Hierarchic pie chart
https://stackoverflow.com/questions/33019879/hierarchic-pie-donut-chart-from-pandas-dataframe-using-bokeh-or-matplotlib

<img src="http://i.stack.imgur.com/YKbzU.png" width="300" />

### Figure 8 Outbreak settings for GI, GII.2, GII.4, GII.6 and GII.17.

## How to create easily
* choose an ORF1_sub type

### Multiple charts
* Use Multichoice or multiselect
* generate all

In [ ]:
# import io
# import random

# from jinja2 import Template

# from bokeh.embed import components
# from bokeh.plotting import figure
# from bokeh.resources import INLINE
# from bokeh.util.browser import view

# ########## BUILD FIGURES ################

# PLOT_OPTIONS = dict(plot_width=800, plot_height=300)
# SCATTER_OPTIONS = dict(size=12, alpha=0.5)

# data = lambda: [random.choice([i for i in range(100)]) for r in range(10)]

# red = figure(sizing_mode='scale_width', tools='pan', **PLOT_OPTIONS)
# red.scatter(data(), data(), color="red", **SCATTER_OPTIONS)

# blue = figure(sizing_mode='fixed', tools='pan', **PLOT_OPTIONS)
# blue.scatter(data(), data(), color="blue", **SCATTER_OPTIONS)

# green = figure(sizing_mode='scale_width', tools='pan', **PLOT_OPTIONS)
# green.scatter(data(), data(), color="green", **SCATTER_OPTIONS)

# ########## RENDER PLOTS ################

# # Define our html template for out plots
# template = Template("""\
# <!DOCTYPE html>
# <html lang="en">
#     <head>
#         <meta charset="utf-8">
#         <title>Responsive plots</title>
#         {{ resources }}
#     </head>
#     <body>
#     <h2>Resize the window to see some plots resizing</h2>
#     <h3>Red - pan tool, scale_width</h3>
#     {{ plot_div.red }}
#     <h3>Green - pan tool, scale_width</h3>
#     {{ plot_div.green }}
#     <h3>Blue - pan tool, fixed/h3>
#     {{ plot_div.blue }}

#     {{ plot_script }}
#     </body>
# </html>
# """)

# resources = INLINE.render()

# script, div = components({'red': red, 'blue': blue, 'green': green})

# html = template.render(resources=resources,
#                        plot_script=script,
#                        plot_div=div)

# filename = 'embed_multiple_responsive.html'

# with io.open(filename, mode='w', encoding='utf-8') as f:
#     f.write(html)

# view(filename)
